In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [2]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(
            base_url = "https://api.fanar.qa/v1",
            api_key  = os.environ["FANAR_API_KEY"],
        )
        client.default_params = {"model": "Fanar-C-1-8.7B"}    
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="fanar"
deployment = APIKeyManager(model_type, "./azure.env")


In [3]:
class TopicClassifier:
    def __init__(self, deployment, model="fanar"):
        self.deployment = deployment
        self.model = model
        
    def classify_topic(self, topic, information=""):
        prompt = f"""Classify this Arabic topic into exactly 3 categories:

Domain: [تقنية/صحة/تعليم/ثقافة/أعمال/عام]
Style: [رسمي/عادي/ودود] 
Sensitivity: [عادي/حساس]

Topic: {topic}
Information: {information}

Return only in this format: Domain|Style|Sensitivity
Example: تقنية|عادي|عادي"""

        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a topic classifier. Return only the requested format."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            result = response.choices[0].message.content.strip()
            return self._parse_classification(result)
            
        except Exception as e:
            print(f"Classification error: {e}")
            return self._get_default_classification()
    
    def _parse_classification(self, result):
        """Parse the LLM result and validate format"""
        try:
            parts = result.split('|')
            if len(parts) == 3:
                domain, style, sensitivity = [part.strip() for part in parts]
                
                # Validate domains
                valid_domains = ['تقنية', 'صحة', 'تعليم', 'ثقافة', 'أعمال', 'عام']
                valid_styles = ['رسمي', 'عادي', 'ودود']
                valid_sensitivity = ['عادي', 'حساس']
                
                if (domain in valid_domains and 
                    style in valid_styles and 
                    sensitivity in valid_sensitivity):
                    
                    return {
                        'domain': domain,
                        'style': style,
                        'sensitivity': sensitivity,
                        'raw_output': result
                    }
            
            # If validation fails, try keyword fallback
            return self._keyword_fallback(result)
            
        except Exception:
            return self._get_default_classification()
    
    def _keyword_fallback(self, text):
        """Simple keyword-based classification as backup"""
        domain = 'عام'  # default
        style = 'عادي'   # default
        sensitivity = 'عادي'  # default
        
        # Simple keyword matching for domain
        if any(word in text for word in ['تقنية', 'تكنولوجيا', 'برمجة', 'ذكاء اصطناعي']):
            domain = 'تقنية'
        elif any(word in text for word in ['صحة', 'طب', 'دواء', 'علاج']):
            domain = 'صحة'
        elif any(word in text for word in ['تعليم', 'تعلم', 'دراسة', 'تربية']):
            domain = 'تعليم'
        elif any(word in text for word in ['ثقافة', 'تراث', 'تاريخ', 'أدب']):
            domain = 'ثقافة'
        elif any(word in text for word in ['أعمال', 'اقتصاد', 'تجارة', 'استثمار']):
            domain = 'أعمال'
            
        return {
            'domain': domain,
            'style': style,
            'sensitivity': sensitivity,
            'raw_output': text,
            'fallback_used': True
        }
    
    def _get_default_classification(self):
        """Default safe classification"""
        return {
            'domain': 'عام',
            'style': 'عادي', 
            'sensitivity': 'عادي',
            'raw_output': 'default',
            'fallback_used': True
        }

# Usage example:
# classifier = TopicClassifier(deployment, "fanar")
# result = classifier.classify_topic("الذكاء الاصطناعي في التعليم", "تأثير التكنولوجيا على طرق التدريس")
# print(result)
# Output: {'domain': 'تقنية', 'style': 'عادي', 'sensitivity': 'عادي', 'raw_output': 'تقنية|عادي|عادي'}

In [4]:
# Usage example:
classifier = TopicClassifier(deployment, "Fanar-C-1-8.7B")
topic = "الذكاء الاصطناعي في التعليم"
information = "تأثير التكنولوجيا على طرق التدريس"
classification = classifier.classify_topic(topic,information)
print(classification)

{'domain': 'تقنية', 'style': 'عادي', 'sensitivity': 'عادي', 'raw_output': 'تقنية|عادي|عادي'}


In [5]:
class PersonaGenerator:
    def __init__(self, deployment, model="fanar"):
        self.deployment = deployment
        self.model = model
        
        # Job templates based on domain
        self.domain_jobs = {
            'تقنية': ['مطور برمجيات', 'مهندس ذكاء اصطناعي', 'خبير أمن سيبراني', 'مؤسس شركة تقنية', 'باحث في علوم الحاسوب'],
            'صحة': ['طبيب متخصص', 'ممرض خبير', 'باحث في الطب', 'أخصائي تغذية', 'مدير مستشفى'],
            'تعليم': ['أستاذ جامعي', 'معلم خبير', 'مدير مدرسة', 'خبير مناهج', 'باحث تربوي'],
            'ثقافة': ['كاتب وأديب', 'مؤرخ', 'ناقد ثقافي', 'فنان', 'باحث في التراث'],
            'أعمال': ['رائد أعمال', 'مدير تنفيذي', 'خبير اقتصادي', 'مستشار مالي', 'محلل استثمار'],
            'عام': ['صحفي', 'إعلامي', 'كاتب', 'متخصص في التنمية', 'استشاري']
        }
    
    def generate_personas(self, classification, topic, information=""):
        """Generate complementary host and guest personas"""
        domain = classification['domain']
        style = classification['style']
        sensitivity = classification['sensitivity']
        
        # Generate host persona (usually more experienced/authoritative)
        host_persona = self._generate_single_persona(
            role="host", 
            domain=domain, 
            style=style, 
            sensitivity=sensitivity,
            topic=topic,
            information=information
        )
        
        # Generate guest persona (complementary to host)
        guest_persona = self._generate_single_persona(
            role="guest", 
            domain=domain, 
            style=style, 
            sensitivity=sensitivity,
            topic=topic,
            information=information,
            host_reference=host_persona
        )
        
        return {
            "host": host_persona,
            "guest": guest_persona,
            "chemistry_note": self._assess_chemistry(host_persona, guest_persona)
        }
    
    def _generate_single_persona(self, role, domain, style, sensitivity, topic, information="", host_reference=None):
        """Generate a single persona using Fanar"""
        
        prompt = f"""Create a {role} persona for Arabic podcast about: {topic}

Return JSON format:
{{
    "age": "Adult",
    "gender": "Male", 
    "OCEAN_Persona": {{
        "O": "High",
        "C": "Medium",
        "E": "High",
        "A": "Medium", 
        "N": "Low"
    }},
    "job_description": "وصف الوظيفة بالعربية",
    "speaking_style": "أسلوب التحدث بالعربية"
}}

Make job relevant to {domain}. Make style {style}."""

        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are an expert in creating authentic Arabic personas. Return only valid JSON."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            result = response.choices[0].message.content.strip()
            return self._parse_persona(result, role, domain)
            
        except Exception as e:
            print(f"Persona generation error: {e}")
            return self._get_fallback_persona(role, domain, style)
    
    def _parse_persona(self, result, role, domain):
        """Parse and validate persona JSON"""
        try:
            # Clean up response (remove markdown if present)
            result = result.replace('```json', '').replace('```', '').strip()
            
            import json
            persona = json.loads(result)
            
            # Validate required fields
            required_fields = ['age', 'gender', 'OCEAN_Persona', 'job_description', 'speaking_style']
            if all(field in persona for field in required_fields):
                persona['role'] = role
                persona['raw_output'] = result
                return persona
            else:
                return self._get_fallback_persona(role, domain, 'عادي')
                
        except Exception:
            return self._get_fallback_persona(role, domain, 'عادي')
    
    def _get_fallback_persona(self, role, domain, style):
        """Generate fallback persona when LLM fails"""
        import random
        
        jobs = self.domain_jobs.get(domain, self.domain_jobs['عام'])
        job = random.choice(jobs)
        
        speaking_styles = {
            'رسمي': 'متحدث بطلاقة ووضوح، يستخدم لغة رسمية ومهنية',
            'عادي': 'متحدث طبيعي ومباشر، يستخدم لغة واضحة ومفهومة', 
            'ودود': 'متحدث دافئ ومرحب، يستخدم لغة ودودة وقريبة من المستمعين'
        }
        
        return {
            "age": "Adult",
            "gender": random.choice(["Male", "Female"]),
            "OCEAN_Persona": {
                "O": "Medium",
                "C": "Medium", 
                "E": "Medium",
                "A": "Medium",
                "N": "Medium"
            },
            "job_description": job,
            "speaking_style": speaking_styles.get(style, speaking_styles['عادي']),
            "role": role,
            "fallback_used": True
        }
    
    def _assess_chemistry(self, host, guest):
        """Simple chemistry assessment between personas"""
        host_e = host['OCEAN_Persona']['E']
        guest_e = guest['OCEAN_Persona']['E']
        
        if host_e == "High" and guest_e == "High":
            return "طاقة عالية - محادثة حيوية ومفعمة بالنشاط"
        elif host_e == "Low" and guest_e == "Low":
            return "طاقة هادئة - محادثة متأنية وعميقة"
        else:
            return "توازن جيد - تفاعل متوازن بين الشخصيتين"

# Usage example:
# persona_generator = PersonaGenerator(deployment, "fanar")
# classification = classifier.classify_topic(topic, information)
# personas = persona_generator.generate_personas(classification, topic, information)

In [6]:
# Usage example:
persona_generator = PersonaGenerator(deployment, "Fanar")
personas = persona_generator.generate_personas(classification, topic, information)
print(personas)

{'host': {'age': 'Adult', 'gender': 'Male', 'OCEAN_Persona': {'O': 'High', 'C': 'Medium', 'E': 'High', 'A': 'Medium', 'N': 'Low'}, 'job_description': 'باحث ذكاء اصطناعي متخصص في تطوير حلول تعليمية مبتكرة', 'speaking_style': 'يتمتع بأسلوب حديث واضح ومباشر، يمزج بين الشرح التقني والتفسيرات البسيطة لجعل المواضيع المعقدة مفهومة للجميع.', 'role': 'host', 'raw_output': '{\n  "age": "Adult",\n  "gender": "Male", \n  "OCEAN_Persona": {\n    "O": "High",\n    "C": "Medium",\n    "E": "High",\n    "A": "Medium", \n    "N": "Low"\n  },\n  "job_description": "باحث ذكاء اصطناعي متخصص في تطوير حلول تعليمية مبتكرة",\n  "speaking_style": "يتمتع بأسلوب حديث واضح ومباشر، يمزج بين الشرح التقني والتفسيرات البسيطة لجعل المواضيع المعقدة مفهومة للجميع."\n}'}, 'guest': {'age': 'Adult', 'gender': 'Male', 'OCEAN_Persona': {'O': 'High', 'C': 'Medium', 'E': 'High', 'A': 'Medium', 'N': 'Low'}, 'job_description': 'مهندس برمجيات ذو خبرة، يعمل على دمج الذكاء الاصطناعي في منصات التعلم الرقمية', 'speaking_style': 'يتمت

In [7]:
class OutlineContentGenerator:
    def __init__(self, deployment, model="fanar"):
        self.deployment = deployment
        self.model = model
    
    def generate_outline(self, topic, information, classification):
        """Generate content-focused outline based on topic and classification"""
        domain = classification['domain']
        style = classification['style']
        
        # Adjust tone based on style
        tone_instructions = {
            'رسمي': 'Use formal, professional tone',
            'عادي': 'Use clear, straightforward tone',
            'ودود': 'Use friendly, welcoming tone'
        }
        tone = tone_instructions.get(style, 'Use clear tone')
        
        prompt = f"""Create a structured podcast outline for this topic: {topic}

Information: {information}
Domain: {domain}
Tone: {tone}

Generate ALL CONTENT in Modern Standard Arabic (MSA). Keep JSON keys in English but all values in Arabic.

Return this exact JSON format:

{{
    "Intro1": {{
        "description": "وصف مختصر للبودكاست والترحيب العام",
        "script": ["بيان افتتاحي عن البودكاست", "رسالة ترحيب للمستمعين"]
    }},
    "Intro2": {{
        "description": "تقديم موضوع اليوم والإعداد",
        "script": ["تقديم موضوع اليوم", "لماذا هذا الموضوع مهم", "ما سيتعلمه المستمعون"]
    }},
    "Points": {{
        "talking_points": {{
            "النقطة الرئيسية الأولى": {{
                "discussion": "نقطة نقاش رئيسية حول الموضوع",
                "questions": [
                    "سؤال ذي صلة؟",
                    "سؤال متابعة؟"
                ],
                "response_hint": "الاتجاه المتوقع للإجابات"
            }},
            "النقطة الرئيسية الثانية": {{
                "discussion": "جانب مهم آخر للمناقشة",
                "questions": [
                    "سؤال ذي صلة آخر؟",
                    "سؤال تفصيلي؟"
                ],
                "response_hint": "الاتجاه المتوقع للإجابات"
            }},
            "النقطة الرئيسية الثالثة": {{
                "discussion": "النقطة الأساسية الأخيرة أو التطبيقات العملية",
                "questions": [
                    "سؤال عملي؟",
                    "سؤال حول التأثيرات المستقبلية؟"
                ],
                "response_hint": "الاتجاه المتوقع للإجابات"
            }}
        }}
    }},
    "Con": {{
        "description": "ملخص وخاتمة مدروسة",
        "script": ["ملخص النقاط الرئيسية", "سؤال ختامي يثير التفكير للجمهور"]
    }}
}}

Generate content that fits {domain} domain. All text content must be in Arabic."""

        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a podcast content planner. Return only valid JSON."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            result = response.choices[0].message.content.strip()
            return self._parse_outline(result, topic, domain)
            
        except Exception as e:
            print(f"Outline generation error: {e}")
            return self._get_fallback_outline(topic, domain)
    
    def print_outline(self, outline):
        """Pretty print the outline in readable format"""
        import json
        print(json.dumps(outline, ensure_ascii=False, indent=2))
    
    def _parse_outline(self, result, topic, domain):
        """Parse and validate outline JSON"""
        try:
            # Clean up response
            result = result.replace('```json', '').replace('```', '').strip()
            
            import json
            outline = json.loads(result)
            
            # Validate structure
            required_sections = ['Intro1', 'Intro2', 'Points', 'Con']
            if all(section in outline for section in required_sections):
                outline['raw_output'] = result
                outline['topic'] = topic
                outline['domain'] = domain
                return outline
            else:
                return self._get_fallback_outline(topic, domain)
                
        except Exception:
            return self._get_fallback_outline(topic, domain)
    
    def _get_fallback_outline(self, topic, domain):
        """Generate simple fallback outline when LLM fails"""
        return {
            "Intro1": {
                "description": "نظرة عامة مختصرة عن البودكاست وترحيب عام",
                "script": [
                    "أهلاً وسهلاً بكم في بودكاستنا",
                    "اليوم نستكشف موضوعاً شيقاً ومهماً"
                ]
            },
            "Intro2": {
                "description": "تقديم موضوع اليوم",
                "script": [
                    f"موضوعنا اليوم هو {topic}",
                    "هذا موضوع مهم يستحق النقاش",
                    "دعونا نتعمق في الجوانب الرئيسية"
                ]
            },
            "Points": {
                "talking_points": {
                    "فهم الأساسيات": {
                        "discussion": f"ما هي المفاهيم الأساسية لـ {topic}؟",
                        "questions": [
                            f"ما هو {topic} بالضبط؟",
                            "لماذا هذا الموضوع مهم اليوم؟"
                        ],
                        "response_hint": "شرح واضح للمفاهيم الأساسية"
                    },
                    "التطبيقات الحالية": {
                        "discussion": f"كيف يتم استخدام {topic} في الممارسة العملية؟",
                        "questions": [
                            "ما هي بعض الأمثلة من الواقع؟",
                            "ما هي الفوائد التي نراها؟"
                        ],
                        "response_hint": "أمثلة عملية وفوائد"
                    },
                    "النظرة المستقبلية": {
                        "discussion": f"ما هو مستقبل {topic}؟",
                        "questions": [
                            "إلى أين ترى هذا المجال متجهاً؟",
                            "ما الذي يجب على الناس متابعته؟"
                        ],
                        "response_hint": "الاتجاهات والآثار المستقبلية"
                    }
                }
            },
            "Con": {
                "description": "ملخص وخاتمة مدروسة",
                "script": [
                    f"النقاط الرئيسية حول {topic} التي يجب تذكرها",
                    "ما الذي يجب أن يفكر فيه مستمعونا بعد هذه المناقشة؟"
                ]
            },
            "fallback_used": True,
            "topic": topic,
            "domain": domain
        }

# Usage example:
# outline_generator = OutlineContentGenerator(deployment, "fanar")
# outline = outline_generator.generate_outline(topic, information, classification)

In [8]:
outline_generator = OutlineContentGenerator(deployment, "Fanar-C-1-8.7B")
outline = outline_generator.generate_outline(topic, information, classification)
outline_generator.print_outline(outline)  # Nice formatting!


{
  "Intro1": {
    "description": "مقدمة قصيرة لبودكاستنا الذي يستكشف دور التكنولوجيا، ومرحباً بجميع المستمعين المهتمين بتأثير التقنية الحديثة.",
    "script": [
      "مرحبًا بكُمْ في بودكاست 'تقنية وتعلم'. سنُناقش اليوم كيف يُغير الذكاء الاصطناعي من ممارسات التعلم والتدريس.",
      "شكراً لكِ/لك على الانضمام إلينا."
    ]
  },
  "Intro2": {
    "description": "إدخال موضوع اليوم - الذكاء الاصطناعي في التعليم - وأهميته وكيف يمكن أن يفيد المستمعين.",
    "script": [
      "اليوم، سنتعمق في كيفية استخدام الذكاء الاصطناعي لتغيير الطرق التي نتعلم بها ونُعلِّم فيها.",
      "هذا الموضوع حيوي لأنّه يتناول مستقبل تعليمنا ويقدم رؤى قيمة لكل المعلمين والمُتعلمين.",
      "من خلال هذه الحلقة، ستفهم أفضل كيفية دمج الذكاء الاصطناعي في بيئة الفصل الدراسي وستكتسب فهمًا أعمق لأثره المحتمل."
    ]
  },
  "Points": {
    "talking_points": {
      "النقطة الرئيسية الأولى": {
        "discussion": "كيف يساعد الذكاء الاصطناعي في تخصيص تجربة التعلم الفردية.",
        "questions": [
          "كيف يمكن للذ

In [9]:
class PersonaEnhancer:
    def __init__(self, deployment, model="fanar"):
        self.deployment = deployment
        self.model = model
    
    def enhance_outline_with_personas(self, outline, host_persona, guest_persona):
        """Enhance generic outline with specific host and guest personas"""
        
        # Extract persona information
        host_job = host_persona['job_description']
        host_style = host_persona['speaking_style']
        guest_job = guest_persona['job_description']
        guest_style = guest_persona['speaking_style']
        
        # Create a simplified version of current outline for the prompt
        current_outline_json = {
            "Intro1": outline.get("Intro1", {}),
            "Intro2": outline.get("Intro2", {}),
            "Points": outline.get("Points", {}),
            "Con": outline.get("Con", {})
        }
        
        prompt = f"""Enhance this podcast outline with the personas provided.

HOST: {host_job} - {host_style}
GUEST: {guest_job} - {guest_style}

CURRENT OUTLINE:
{self._format_outline_simple(current_outline_json)}

Enhance it by:
1. Modify introductions to mention the personas
2. Adjust questions to match host expertise
3. Update response hints for guest background
4. Keep all content in Arabic

Return the EXACT same JSON structure:

{{
    "Intro1": {{
        "description": "وصف بالعربية",
        "script": ["نص بالعربية"]
    }},
    "Intro2": {{
        "description": "وصف بالعربية", 
        "script": ["نص بالعربية"]
    }},
    "Points": {{
        "talking_points": {{
            "عنوان النقطة": {{
                "discussion": "النقاش بالعربية",
                "questions": ["سؤال بالعربية؟"],
                "response_hint": "توجيه الإجابة بالعربية"
            }}
        }}
    }},
    "Con": {{
        "description": "وصف بالعربية",
        "script": ["نص بالعربية"]
    }}
}}"""

        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only the requested JSON format. Do not add extra structure or wrapper objects."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            result = response.choices[0].message.content.strip()
            return self._parse_enhanced_outline(result, outline)
            
        except Exception as e:
            print(f"Persona enhancement error: {e}")
            return self._get_persona_enhanced_fallback(outline, host_persona, guest_persona)
    
    def _format_outline_simple(self, outline):
        """Simple outline formatting for prompt"""
        import json
        return json.dumps(outline, ensure_ascii=False, indent=2)
    
    def _format_outline_for_prompt(self, outline):
        """Format outline for prompt (simplified version)"""
        formatted = ""
        
        # Add Intro sections
        if 'Intro1' in outline:
            formatted += f"Intro1: {outline['Intro1']['description']}\n"
            formatted += f"Scripts: {outline['Intro1']['script']}\n\n"
        
        if 'Intro2' in outline:
            formatted += f"Intro2: {outline['Intro2']['description']}\n"
            formatted += f"Scripts: {outline['Intro2']['script']}\n\n"
        
        # Add talking points
        if 'Points' in outline and 'talking_points' in outline['Points']:
            formatted += "Main Discussion Points:\n"
            for point_title, point_content in outline['Points']['talking_points'].items():
                formatted += f"- {point_title}: {point_content['discussion']}\n"
                formatted += f"  Questions: {point_content['questions']}\n"
                formatted += f"  Response hint: {point_content['response_hint']}\n\n"
        
        # Add conclusion
        if 'Con' in outline:
            formatted += f"Conclusion: {outline['Con']['description']}\n"
            formatted += f"Scripts: {outline['Con']['script']}\n"
        
        return formatted
    
    def _parse_enhanced_outline(self, result, original_outline):
        """Parse and validate enhanced outline"""
        print("=== DEBUGGING PERSONA ENHANCEMENT ===")
        print(f"Raw result length: {len(result)}")
        print(f"Raw result (first 200 chars): {result[:200]}...")
        print(f"Raw result (last 200 chars): ...{result[-200:]}")
        
        try:
            # Clean up response
            result_cleaned = result.replace('```json', '').replace('```', '').strip()
            print(f"After cleaning: {result_cleaned[:100]}...")
            
            import json
            enhanced_outline = json.loads(result_cleaned)
            print(f"JSON parsing successful!")
            print(f"Top-level keys found: {list(enhanced_outline.keys())}")
            
            # Validate structure matches original
            required_sections = ['Intro1', 'Intro2', 'Points', 'Con']
            missing_sections = [section for section in required_sections if section not in enhanced_outline]
            
            print(f"Required sections: {required_sections}")
            print(f"Missing sections: {missing_sections}")
            
            if len(missing_sections) == 0:
                print("✅ All required sections found - SUCCESS!")
                enhanced_outline['persona_enhanced'] = True
                enhanced_outline['raw_enhancement'] = result
                return enhanced_outline
            else:
                print(f"❌ Validation failed - missing sections: {missing_sections}")
                print("Using fallback...")
                return self._get_persona_enhanced_fallback(original_outline, None, None)
                
        except json.JSONDecodeError as e:
            print(f"❌ JSON parsing failed: {e}")
            print("Using fallback...")
            return self._get_persona_enhanced_fallback(original_outline, None, None)
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            print("Using fallback...")
            return self._get_persona_enhanced_fallback(original_outline, None, None)
    
    def _get_persona_enhanced_fallback(self, original_outline, host_persona, guest_persona):
        """Simple persona enhancement fallback"""
        enhanced = original_outline.copy()
        
        if host_persona and guest_persona:
            # Simple modifications to show persona influence
            host_job = host_persona.get('job_description', 'مقدم برامج')
            guest_job = guest_persona.get('job_description', 'ضيف متخصص')
            
            # Modify Intro2 to mention personas
            if 'Intro2' in enhanced:
                enhanced['Intro2']['script'].append(f"معنا اليوم {guest_job}")
                enhanced['Intro2']['script'].append(f"وسيقدم الحلقة {host_job}")
            
            # Add persona context to first talking point
            if 'Points' in enhanced and 'talking_points' in enhanced['Points']:
                points = list(enhanced['Points']['talking_points'].keys())
                if points:
                    first_point = points[0]
                    enhanced['Points']['talking_points'][first_point]['response_hint'] += f" من منظور {guest_job}"
        
        enhanced['persona_enhanced'] = True
        enhanced['fallback_used'] = True
        return enhanced
    
    def print_enhanced_outline(self, enhanced_outline):
        """Pretty print the enhanced outline"""
        import json
        print("=== PERSONA-ENHANCED OUTLINE ===")
        print(json.dumps(enhanced_outline, ensure_ascii=False, indent=2))
        
        if enhanced_outline.get('persona_enhanced'):
            if enhanced_outline.get('fallback_used'):
                print("\n⚠️ Note: Fallback enhancement was used")
            else:
                print("\n✅ Successfully enhanced with personas")

# Usage example:
# persona_enhancer = PersonaEnhancer(deployment, "fanar")
# enhanced_outline = persona_enhancer.enhance_outline_with_personas(outline, host_persona, guest_persona)
# persona_enhancer.print_enhanced_outline(enhanced_outline)

In [10]:
# Create enhancer
persona_enhancer = PersonaEnhancer(deployment, "Fanar-C-1-8.7B")

# Enhance your outline with personas
enhanced_outline = persona_enhancer.enhance_outline_with_personas(
    outline, 
    personas['host'], 
    personas['guest']
)

# Pretty print results
persona_enhancer.print_enhanced_outline(enhanced_outline)

=== DEBUGGING PERSONA ENHANCEMENT ===
Raw result length: 3442
Raw result (first 200 chars): بالتأكيد! إليك نسخة مُحسَّنة من مخطط البودكاست الخاص بك بما يتماشى مع طلبك:

```json
{
  "Intro1": {
    "description": "مقدمة قصيرة لبودكاستنا الذي يستكشف تأثير التكنولوجيا الحديثة في التعليم, حيث سي...
Raw result (last 200 chars): ...قوة تغييرية في مجال التعليم.",
      "دعونا نتذكر أهمية توازن الإيجابيات والسلبيات عند تنفيذه.",
      "ما هو رأيك في مستقبل الذكاء الاصطناعي في التعليم? دعنا نسمع وجهة نظرك كمستمعين!"
    ]
  }
}
```
After cleaning: بالتأكيد! إليك نسخة مُحسَّنة من مخطط البودكاست الخاص بك بما يتماشى مع طلبك:


{
  "Intro1": {
    "d...
❌ JSON parsing failed: Expecting value: line 1 column 1 (char 0)
Using fallback...
=== PERSONA-ENHANCED OUTLINE ===
{
  "Intro1": {
    "description": "مقدمة قصيرة لبودكاستنا الذي يستكشف دور التكنولوجيا، ومرحباً بجميع المستمعين المهتمين بتأثير التقنية الحديثة.",
    "script": [
      "مرحبًا بكُمْ في بودكاست 'تقنية وتعلم'. سنُناقش اليوم كيف يُغير ال

In [11]:
class CulturalEnhancer:
    def __init__(self, deployment, model="fanar"):
        self.deployment = deployment
        self.model = model
        
        # Cultural expressions templates
        self.cultural_templates = {
            'opening': ['بسم الله نبدأ', 'على بركة الله نستهل', 'بإذن الله نبدأ'],
            'transitions': ['وهذا يقودنا إلى', 'ومن هنا ننتقل إلى', 'وبهذا نصل إلى'],
            'gratitude': ['نشكركم على هذا التوضيح', 'جزاكم الله خيراً', 'بارك الله فيكم'],
            'respect': ['الأستاذ الفاضل', 'الدكتور المحترم', 'أستاذنا الكريم'],
            'closing': ['بارك الله في علمكم', 'نسأل الله التوفيق', 'والله أعلم']
        }
    
    def add_cultural_context(self, enhanced_outline, classification):
        """Add cultural elements using micro-LLM calls for each individual element"""
        
        sensitivity = classification.get('sensitivity', 'عادي')
        culturally_enhanced = enhanced_outline.copy()
        
        try:
            # 1. Enhance FIRST script in Intro1 only
            if 'Intro1' in culturally_enhanced and 'script' in culturally_enhanced['Intro1']:
                scripts = culturally_enhanced['Intro1']['script']
                if scripts:
                    # Find main welcome script
                    for i, script in enumerate(scripts):
                        if 'مرحب' in script and len(script) > 20:
                            enhanced_script = self._enhance_single_text(
                                script, 
                                "Add 'بسم الله نبدأ،' at the beginning"
                            )
                            scripts[i] = enhanced_script
                            print(f"✅ Enhanced Intro1 script")
                            break
            
            # 2. Enhance guest introduction in Intro2
            if 'Intro2' in culturally_enhanced and 'script' in culturally_enhanced['Intro2']:
                scripts = culturally_enhanced['Intro2']['script']
                for i, script in enumerate(scripts):
                    if 'معنا اليوم' in script:
                        enhanced_script = self._enhance_single_text(
                            script,
                            "Add 'الأستاذ الفاضل' after 'معنا اليوم'"
                        )
                        scripts[i] = enhanced_script
                        print(f"✅ Enhanced Intro2 guest introduction")
                        break
            
            # 3. Enhance EACH talking point discussion separately
            if ('Points' in culturally_enhanced and 
                'talking_points' in culturally_enhanced['Points']):
                
                talking_points = culturally_enhanced['Points']['talking_points']
                point_keys = list(talking_points.keys())
                
                # Only enhance FIRST point with transition
                if point_keys:
                    first_key = point_keys[0]
                    first_point = talking_points[first_key]
                    
                    if 'discussion' in first_point:
                        enhanced_discussion = self._enhance_single_text(
                            first_point['discussion'],
                            "Add 'وهذا يقودنا إلى' at the beginning"
                        )
                        first_point['discussion'] = enhanced_discussion
                        print(f"✅ Enhanced first discussion point")
            
            # 4. Enhance conclusion - create clean closing
            if 'Con' in culturally_enhanced and 'script' in culturally_enhanced['Con']:
                scripts = culturally_enhanced['Con']['script']
                
                # Find the main closing statement
                main_closing = None
                for script in scripts:
                    if ('خلاصة' in script or 'ختتم' in script) and len(script) > 30:
                        main_closing = script
                        break
                
                if main_closing:
                    # Enhance the main closing with gratitude
                    enhanced_closing = self._enhance_single_text(
                        main_closing,
                        "Add 'نشكركم،' at the beginning and 'بارك الله فيكم' at the end"
                    )
                    
                    # Create clean script array
                    new_scripts = [enhanced_closing]
                    if sensitivity == 'حساس':
                        new_scripts.append("والله أعلم.")
                    
                    culturally_enhanced['Con']['script'] = new_scripts
                    print(f"✅ Enhanced conclusion")
            
            culturally_enhanced['culturally_enhanced'] = True
            print("✅ All cultural enhancements completed successfully!")
            return culturally_enhanced
            
        except Exception as e:
            print(f"Cultural enhancement error: {e}")
            return self._add_cultural_fallback(enhanced_outline, sensitivity)
    
    def _enhance_single_text(self, original_text, instruction):
        """Enhance a single piece of text with simple instruction"""
        
        prompt = f"""{instruction}

Original: {original_text}

Return only the enhanced text."""
        
        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only the enhanced text. No explanations, no quotes, no extra text."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            enhanced = response.choices[0].message.content.strip()
            
            # Clean up any quotes or extra formatting
            enhanced = enhanced.strip('"').strip("'").strip()
            
            return enhanced
            
        except Exception as e:
            print(f"❌ Text enhancement failed: {e}")
            # Fallback to simple manual enhancement
            if "بسم الله نبدأ" in instruction:
                return f"بسم الله نبدأ، {original_text}"
            elif "الأستاذ الفاضل" in instruction:
                return original_text.replace("معنا اليوم", "معنا اليوم الأستاذ الفاضل")
            elif "وهذا يقودنا إلى" in instruction:
                return f"وهذا يقودنا إلى {original_text}"
            elif "نشكركم" in instruction and "بارك الله فيكم" in instruction:
                return f"نشكركم، {original_text} بارك الله فيكم."
            else:
                return original_text
    
    def _enhance_intro1(self, intro1_section):
        """Integrate opening blessing into the first script naturally"""
        prompt = f"""Modify the first script line to include "بسم الله نبدأ" naturally at the beginning.

Current first script: "{intro1_section['script'][0] if intro1_section.get('script') else ''}"

Make it: "بسم الله نبدأ، [rest of the script]"

Return only the enhanced script text, not full JSON."""
        
        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only the enhanced script text. No JSON, no explanations."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            enhanced_script = response.choices[0].message.content.strip().strip('"')
            
            # Apply the enhancement
            enhanced = intro1_section.copy()
            if 'script' in enhanced and enhanced['script']:
                enhanced['script'][0] = enhanced_script
            print("✅ Intro1 enhanced successfully")
            return enhanced
            
        except Exception as e:
            print(f"❌ Intro1 enhancement failed: {e}")
            # Simple fallback - integrate manually
            enhanced = intro1_section.copy()
            if 'script' in enhanced and enhanced['script']:
                current = enhanced['script'][0]
                enhanced['script'][0] = f"بسم الله نبدأ، {current}"
            return enhanced
    
    def _enhance_intro2(self, intro2_section):
        """Integrate respectful address naturally into guest introduction"""
        prompt = f"""Find the script line that introduces the guest and add "الأستاذ الفاضل" naturally.

Current scripts: {intro2_section.get('script', [])}

Modify the line that says "معنا اليوم" to include "الأستاذ الفاضل" naturally.

Return only the modified script line, not full JSON."""
        
        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only the enhanced script text. No JSON, no explanations."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            enhanced_script = response.choices[0].message.content.strip().strip('"')
            
            # Apply the enhancement
            enhanced = intro2_section.copy()
            if 'script' in enhanced:
                for i, script in enumerate(enhanced['script']):
                    if 'معنا اليوم' in script:
                        enhanced['script'][i] = enhanced_script
                        break
            print("✅ Intro2 enhanced successfully")
            return enhanced
            
        except Exception as e:
            print(f"❌ Intro2 enhancement failed: {e}")
            # Simple fallback
            enhanced = intro2_section.copy()
            if 'script' in enhanced:
                for i, script in enumerate(enhanced['script']):
                    if 'معنا اليوم' in script and 'الأستاذ الفاضل' not in script:
                        enhanced['script'][i] = script.replace('معنا اليوم', 'معنا اليوم الأستاذ الفاضل')
                        break
            return enhanced
    
    def _enhance_points(self, points_section):
        """Integrate transition phrase naturally into first discussion"""
        if 'talking_points' not in points_section:
            return points_section
            
        points = list(points_section['talking_points'].keys())
        if not points:
            return points_section
            
        first_point = points[0]
        current_discussion = points_section['talking_points'][first_point]['discussion']
        
        prompt = f"""Add "وهذا يقودنا إلى" naturally at the beginning of this discussion.

Current discussion: "{current_discussion}"

Make it flow naturally: "وهذا يقودنا إلى [rest of discussion]"

Return only the enhanced discussion text, not full JSON."""
        
        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only the enhanced discussion text. No JSON, no explanations."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            enhanced_discussion = response.choices[0].message.content.strip().strip('"')
            
            # Apply the enhancement
            enhanced = points_section.copy()
            enhanced['talking_points'][first_point]['discussion'] = enhanced_discussion
            print("✅ Points enhanced successfully")
            return enhanced
            
        except Exception as e:
            print(f"❌ Points enhancement failed: {e}")
            # Simple fallback
            enhanced = points_section.copy()
            enhanced['talking_points'][first_point]['discussion'] = f"وهذا يقودنا إلى {current_discussion}"
            return enhanced
    
    def _enhance_conclusion(self, con_section, sensitivity):
        """Integrate gratitude and blessing naturally into existing conclusion"""
        if not con_section.get('script'):
            return con_section
            
        # Enhance the last script with gratitude and blessing
        last_script = con_section['script'][-1]
        
        extra_blessing = " والله أعلم" if sensitivity == 'حساس' else ""
        
        prompt = f"""Add gratitude "نشكركم" and blessing "بارك الله فيكم" naturally to this closing.

Current closing: "{last_script}"

Integrate them naturally into the text, don't add as separate sentences.{extra_blessing}

Return only the enhanced closing text, not full JSON."""
        
        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Return only the enhanced closing text. No JSON, no explanations."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            enhanced_closing = response.choices[0].message.content.strip().strip('"')
            
            # Apply the enhancement
            enhanced = con_section.copy()
            enhanced['script'][-1] = enhanced_closing
            print("✅ Conclusion enhanced successfully")
            return enhanced
            
        except Exception as e:
            print(f"❌ Conclusion enhancement failed: {e}")
            # Simple fallback
            enhanced = con_section.copy()
            enhanced['script'][-1] = f"نشكركم، {last_script} بارك الله فيكم{extra_blessing}."
            return enhanced
    
    def _parse_cultural_outline(self, result, original_outline):
        """Parse culturally enhanced outline"""
        print("=== DEBUGGING CULTURAL ENHANCEMENT ===")
        print(f"Raw result length: {len(result)}")
        print(f"Raw result (first 300 chars): {result[:300]}...")
        print(f"Raw result (last 200 chars): ...{result[-200:]}")
        
        try:
            # Clean response
            result_cleaned = result.replace('```json', '').replace('```', '').strip()
            print(f"After cleaning (first 200 chars): {result_cleaned[:200]}...")
            
            import json
            cultural_outline = json.loads(result_cleaned)
            print(f"JSON parsing successful!")
            print(f"Top-level keys found: {list(cultural_outline.keys())}")
            
            # Validate structure
            required_sections = ['Intro1', 'Intro2', 'Points', 'Con']
            missing_sections = [section for section in required_sections if section not in cultural_outline]
            
            print(f"Required sections: {required_sections}")
            print(f"Missing sections: {missing_sections}")
            
            if len(missing_sections) == 0:
                print("✅ All required sections found - CULTURAL SUCCESS!")
                cultural_outline['culturally_enhanced'] = True
                cultural_outline['raw_cultural'] = result
                return cultural_outline
            else:
                print(f"❌ Cultural validation failed - missing sections: {missing_sections}")
                print("Using cultural fallback...")
                return self._add_cultural_fallback(original_outline, 'عادي')
                
        except json.JSONDecodeError as e:
            print(f"❌ Cultural JSON parsing failed: {e}")
            print("Using cultural fallback...")
            return self._add_cultural_fallback(original_outline, 'عادي')
        except Exception as e:
            print(f"❌ Unexpected cultural error: {e}")
            print("Using cultural fallback...")
            return self._add_cultural_fallback(original_outline, 'عادي')
    
    def _add_cultural_fallback(self, outline, sensitivity):
        """Simple rule-based cultural enhancement fallback"""
        import random
        
        enhanced = outline.copy()
        
        # Add cultural opening to Intro1
        if 'Intro1' in enhanced and 'script' in enhanced['Intro1']:
            opening = random.choice(self.cultural_templates['opening'])
            enhanced['Intro1']['script'].insert(0, f"{opening}.")
        
        # Add respectful address to Intro2  
        if 'Intro2' in enhanced and 'script' in enhanced['Intro2']:
            respect = random.choice(self.cultural_templates['respect'])
            for i, script in enumerate(enhanced['Intro2']['script']):
                if 'معنا اليوم' in script:
                    enhanced['Intro2']['script'][i] = script.replace('معنا اليوم', f'معنا اليوم {respect}')
                    break
        
        # Add transition to first talking point
        if 'Points' in enhanced and 'talking_points' in enhanced['Points']:
            points = list(enhanced['Points']['talking_points'].keys())
            if points:
                first_point = points[0]
                transition = random.choice(self.cultural_templates['transitions'])
                current_discussion = enhanced['Points']['talking_points'][first_point]['discussion']
                enhanced['Points']['talking_points'][first_point]['discussion'] = f"{transition} {current_discussion}"
        
        # Add cultural closing to Con
        if 'Con' in enhanced and 'script' in enhanced['Con']:
            gratitude = random.choice(self.cultural_templates['gratitude'])
            closing = random.choice(self.cultural_templates['closing'])
            
            enhanced['Con']['script'].insert(0, f"{gratitude} على هذه المناقشة الثرية.")
            enhanced['Con']['script'].append(f"{closing}.")
            
            # Add extra religious expression for sensitive topics
            if sensitivity == 'حساس':
                enhanced['Con']['script'].append("والله أعلم.")
        
        enhanced['culturally_enhanced'] = True
        enhanced['fallback_cultural'] = True
        return enhanced
    
    def print_cultural_outline(self, cultural_outline):
        """Pretty print the culturally enhanced outline"""
        import json
        print("=== CULTURALLY ENHANCED OUTLINE ===")
        print(json.dumps(cultural_outline, ensure_ascii=False, indent=2))
        
        if cultural_outline.get('culturally_enhanced'):
            if cultural_outline.get('fallback_cultural'):
                print("\n⚠️ Note: Fallback cultural enhancement was used")
            else:
                print("\n✅ Successfully enhanced with cultural context")

# Usage example:
# cultural_enhancer = CulturalEnhancer(deployment, "fanar")
# final_outline = cultural_enhancer.add_cultural_context(enhanced_outline, classification)
# cultural_enhancer.print_cultural_outline(final_outline)

In [12]:
cultural_enhancer = CulturalEnhancer(deployment, "Fanar-C-1-8.7B")
final_outline = cultural_enhancer.add_cultural_context(enhanced_outline, classification)
cultural_enhancer.print_cultural_outline(final_outline)

✅ Enhanced Intro1 script
✅ Enhanced first discussion point
✅ Enhanced conclusion
✅ All cultural enhancements completed successfully!
=== CULTURALLY ENHANCED OUTLINE ===
{
  "Intro1": {
    "description": "مقدمة قصيرة لبودكاستنا الذي يستكشف دور التكنولوجيا، ومرحباً بجميع المستمعين المهتمين بتأثير التقنية الحديثة.",
    "script": [
      "بسم الله نبدأ، مرحبًا بكُمْ في بودكاست 'تقنية وتعلم'.",
      "شكراً لكِ/لك على الانضمام إلينا."
    ]
  },
  "Intro2": {
    "description": "إدخال موضوع اليوم - الذكاء الاصطناعي في التعليم - وأهميته وكيف يمكن أن يفيد المستمعين.",
    "script": [
      "اليوم، سنتعمق في كيفية استخدام الذكاء الاصطناعي لتغيير الطرق التي نتعلم بها ونُعلِّم فيها.",
      "هذا الموضوع حيوي لأنّه يتناول مستقبل تعليمنا ويقدم رؤى قيمة لكل المعلمين والمُتعلمين.",
      "من خلال هذه الحلقة، ستفهم أفضل كيفية دمج الذكاء الاصطناعي في بيئة الفصل الدراسي وستكتسب فهمًا أعمق لأثره المحتمل."
    ]
  },
  "Points": {
    "talking_points": {
      "النقطة الرئيسية الأولى": {
        "discu

In [13]:
# class ScriptGenerator:
#     def __init__(self, deployment, model="fanar"):
#         self.deployment = deployment
#         self.model = model
        
#         # Conversational fillers for natural dialogue
#         self.fillers_instruction = """استخدم حشو المحادثة الطبيعي بكثافة متوسطة:
# - تفكير: اممم، اههه، يعني كيف أقول، خلاص
# - تأكيد: طبعاً، تماماً، بالضبط، صحيح
# - تردد: بعنييييي، يعني، اه ما أدري، شوف
# - انفعال: واو، يا الله، ما شاء الله، الله يعطيك العافية
# - ربط: بس، لكن، وبعدين، يا أخي، اسمع
# - خليجي خفيف: شلون، وش رايك، زين، ماشي الحال"""
    
#     def generate_full_script(self, enhanced_outline, personas, classification):
#         """Generate complete podcast script section by section"""
        
#         host_name = personas['host']['job_description']
#         guest_name = personas['guest']['job_description']
        
#         scripts = {}
        
#         try:
#             # 1. Generate Intro1 script
#             if 'Intro1' in enhanced_outline:
#                 scripts['Intro1'] = self._generate_intro1_script(
#                     enhanced_outline['Intro1'], host_name
#                 )
#                 print("✅ Intro1 script generated")
            
#             # 2. Generate Intro2 script  
#             if 'Intro2' in enhanced_outline:
#                 scripts['Intro2'] = self._generate_intro2_script(
#                     enhanced_outline['Intro2'], host_name, guest_name
#                 )
#                 print("✅ Intro2 script generated")
            
#             # 3. Generate Points scripts (one by one)
#             if 'Points' in enhanced_outline and 'talking_points' in enhanced_outline['Points']:
#                 scripts['Points'] = self._generate_points_scripts(
#                     enhanced_outline['Points']['talking_points'], host_name, guest_name
#                 )
#                 print("✅ Points scripts generated")
            
#             # 4. Generate Conclusion script
#             if 'Con' in enhanced_outline:
#                 scripts['Con'] = self._generate_conclusion_script(
#                     enhanced_outline['Con'], host_name, guest_name, classification
#                 )
#                 print("✅ Conclusion script generated")
            
#             print("✅ All scripts generated successfully!")
#             return scripts
            
#         except Exception as e:
#             print(f"Script generation error: {e}")
#             return self._get_fallback_scripts(enhanced_outline, host_name, guest_name)
    
#     def _generate_intro1_script(self, intro1_section, host_name):
#         """Generate opening script with cultural blessing"""
        
#         content = intro1_section.get('description', '') + " " + " ".join(intro1_section.get('script', []))
        
#         prompt = f"""Create opening dialogue for Arabic podcast.

# Content: {content}
# Host: {host_name}

# Include:
# - بسم الله نبدأ naturally
# - Warm welcome 
# - Brief podcast introduction

# {self.fillers_instruction}

# Format:
# المقدم: [dialogue here]

# Return only the script."""
        
#         return self._call_fanar_for_script(prompt, "opening script")
    
#     def _generate_intro2_script(self, intro2_section, host_name, guest_name):
#         """Generate topic introduction with guest introduction"""
        
#         content = intro2_section.get('description', '') + " " + " ".join(intro2_section.get('script', []))
        
#         prompt = f"""Create topic introduction dialogue.

# Content: {content}
# Host: {host_name}
# Guest: {guest_name}

# Include:
# - Topic introduction
# - الأستاذ الفاضل when introducing guest
# - Natural conversation flow

# {self.fillers_instruction}

# Format:
# المقدم: [dialogue]
# الضيف: [response]

# Return only the script."""
        
#         return self._call_fanar_for_script(prompt, "intro2 script")
    
#     def _generate_points_scripts(self, talking_points, host_name, guest_name):
#         """Generate scripts for each talking point separately"""
        
#         point_scripts = {}
        
#         for point_title, point_content in talking_points.items():
#             script = self._generate_single_point_script(
#                 point_title, point_content, host_name, guest_name
#             )
#             point_scripts[point_title] = script
#             print(f"✅ Generated script for: {point_title[:30]}...")
        
#         return point_scripts
    
#     def _generate_single_point_script(self, point_title, point_content, host_name, guest_name):
#         """Generate script for one talking point"""
        
#         discussion = point_content.get('discussion', '')
#         questions = point_content.get('questions', [])
#         response_hint = point_content.get('response_hint', '')
        
#         prompt = f"""Create dialogue for one discussion point.

# Point: {point_title}
# Discussion: {discussion}
# Questions: {questions}
# Response guide: {response_hint}

# Host: {host_name}
# Guest: {guest_name}

# Include:
# - وهذا يقودنا إلى for transitions
# - Natural question-answer flow
# - Expert insights from both personas

# {self.fillers_instruction}

# Format:
# المقدم: [question/discussion]
# الضيف: [expert response]
# المقدم: [follow-up]
# الضيف: [detailed answer]

# Return only the script."""
        
#         return self._call_fanar_for_script(prompt, f"point script: {point_title[:20]}")
    
#     def _generate_conclusion_script(self, con_section, host_name, guest_name, classification):
#         """Generate conclusion with cultural closing"""
        
#         content = con_section.get('description', '') + " " + " ".join(con_section.get('script', []))
#         sensitivity = classification.get('sensitivity', 'عادي')
        
#         extra_blessing = "والله أعلم" if sensitivity == 'حساس' else ""
        
#         prompt = f"""Create conclusion dialogue.

# Content: {content}
# Host: {host_name}
# Guest: {guest_name}

# Include:
# - نشكركم for gratitude
# - بارك الله فيكم for blessing
# - Thoughtful closing question
# {f"- {extra_blessing} for sensitive topics" if extra_blessing else ""}

# {self.fillers_instruction}

# Format:
# المقدم: [summary and thanks]
# الضيف: [final thoughts]
# المقدم: [closing blessing]

# Return only the script."""
        
#         return self._call_fanar_for_script(prompt, "conclusion script")
    
#     def _call_fanar_for_script(self, prompt, script_type):
#         """Call Fanar to generate script"""
        
#         try:
#             response = self.deployment.chat.completions.create(
#                 model=self.model,
#                 messages=[
#                     {"role": "system", "content": "Generate natural Arabic podcast dialogue. Return only the script in the requested format."},
#                     {"role": "user", "content": prompt}
#                 ]
#             )
            
#             script = response.choices[0].message.content.strip()
#             return script
            
#         except Exception as e:
#             print(f"❌ {script_type} generation failed: {e}")
#             return f"المقدم: [فشل في توليد النص لـ {script_type}]"
    
#     def _get_fallback_scripts(self, enhanced_outline, host_name, guest_name):
#         """Simple fallback scripts if everything fails"""
        
#         return {
#             'Intro1': "المقدم: بسم الله نبدأ، مرحباً بكم في بودكاستنا.",
#             'Intro2': f"المقدم: معنا اليوم الأستاذ الفاضل {guest_name}.\nالضيف: أشكركم على الاستضافة.",
#             'Points': {"نقطة عامة": "المقدم: وهذا يقودنا إلى سؤال مهم...\nالضيف: طبعاً، هذا موضوع مهم جداً."},
#             'Con': f"المقدم: نشكركم على هذه المناقشة الثرية.\nالضيف: شكراً لكم.\nالمقدم: بارك الله فيكم."
#         }
    
#     def print_scripts(self, scripts):
#         """Pretty print all generated scripts"""
        
#         print("\n" + "="*50)
#         print("GENERATED PODCAST SCRIPTS")
#         print("="*50)
        
#         for section, script in scripts.items():
#             print(f"\n--- {section} ---")
#             if isinstance(script, dict):  # Points section
#                 for point_title, point_script in script.items():
#                     print(f"\n** {point_title} **")
#                     print(point_script)
#             else:
#                 print(script)
        
#         print("\n" + "="*50)

# # Usage example:
# # script_generator = ScriptGenerator(deployment, "fanar")
# # scripts = script_generator.generate_full_script(enhanced_outline, personas, classification)
# # script_generator.print_scripts(scripts)

In [18]:
class ScriptGenerator:
    def __init__(self, deployment, model="fanar"):
        self.deployment = deployment
        self.model = model
        self.context_history = []
        
        # Real podcast examples for few-shot learning
        self.examples = {
            'intro1': """المقدم: أهلا بكم معنا، اليوم رح نتكلم عن موضوع يعني مذهل شوي، الحوسبة الكمومية
المقدم: ورح نحاول نفهم أساسياتها سوا وكيف تختلف عن الكمبيوترات اللي نستخدمها والإمكانيات الرهيبة اللي ممكن تفتحها
المقدم: يقولون إنها ممكن تحل مسائل، أحتاج حواسيبنا الخارقة اليوم ملايين السنين، كيف ممكن هذا؟
المقدم: القصة بتبدأ من فكرة بصراحة كأنها خيال علمي""",
            
            'intro2': """الضيف: بدل ما نعتمد على الفيزياء العادية، الحواسيب دي بتدخل لعالم ميكانيكا الكم الغريب
الضيف: وبتستخدم دواهر زي التراكب وتشابك عشان تحسب
الضيف: طريقة تفكير مختلفة تماما عن معالجة المعلومات
المقدم: طيب، خلينا نبدأ بالفرق الجوهري""",
            
            'point': """المقدم: كمبيوتراتنا تستخدم البت، يا صفري يا واحد، بسيطة
المقدم: بس الحوسبة الكمومية بتستخدم الكيوبت
المقدم: ايش اللي يخلي الكيوبت ده مختلف لها الدرجة؟
الضيف: هنا بقى القوة والغرابة كلها
الضيف: الكيوبت مش بس صفر أو واحد
الضيف: بفضل التراكب الكمومي هو يقدر يكون صفر وواحد في نفس الوقت أو أي نسبة منهم""",
            
            'conclusion': """المقدم: إذن الخلاصة هي إننا نستخدم أغرب جوانب الفيزياء عشان نبتكر طريقة حساب جديدة كلياً
المقدم: طريقة ممكن تغير قواعد اللعبة في مجالات كثيرة
الضيف: تماماً كده
المقدم: الانتقال من البت للكيوبت مش مجرد سرعة زيادة
المقدم: لا دي قفزة نوعية في قدرات المعالجة"""
        }
    
    def generate_full_script(self, enhanced_outline, personas, classification):
        """Generate complete podcast script section by section with context flow"""
        
        host_name = "المقدم"
        guest_name = "الضيف"
        
        scripts = {}
        self.context_history = []
        
        try:
            # 1. Generate Intro1 script
            if 'Intro1' in enhanced_outline:
                scripts['Intro1'] = self._generate_intro1_script(
                    enhanced_outline['Intro1'], host_name
                )
                self.context_history.append(f"Intro1: {scripts['Intro1'][:100]}...")
                print("✅ Intro1 script generated")
            
            # 2. Generate Intro2 script
            if 'Intro2' in enhanced_outline:
                scripts['Intro2'] = self._generate_intro2_script(
                    enhanced_outline['Intro2'], host_name, guest_name
                )
                self.context_history.append(f"Intro2: {scripts['Intro2'][:100]}...")
                print("✅ Intro2 script generated")
            
            # 3. Generate Points scripts
            if 'Points' in enhanced_outline and 'talking_points' in enhanced_outline['Points']:
                scripts['Points'] = self._generate_points_scripts(
                    enhanced_outline['Points']['talking_points'], host_name, guest_name
                )
                print("✅ Points scripts generated")
            
            # 4. Generate Conclusion script
            if 'Con' in enhanced_outline:
                scripts['Con'] = self._generate_conclusion_script(
                    enhanced_outline['Con'], host_name, guest_name, classification
                )
                print("✅ Conclusion script generated")
            
            print("✅ All scripts generated successfully!")
            return scripts
            
        except Exception as e:
            print(f"Script generation error: {e}")
            return self._get_fallback_scripts(enhanced_outline, host_name, guest_name)
    
    def _generate_intro1_script(self, intro1_section, host_name):
        """Generate brief opening script"""
        
        content = intro1_section.get('description', '') + " " + " ".join(intro1_section.get('script', []))
        
        prompt = f"""Generate a natural podcast opening (2-3 very short turns).

Topic: {content}

Style example (don't copy, use as reference only):
{self.examples['intro1']}

Requirements:
- Start with natural welcome
- Introduce topic in an engaging way
- Each turn: MAX 1 sentence, MAX 20 words
- Use natural Arabic fillers like: يعني، شوي، بصراحة، رح نحاول

Generate the script:"""
        
        return self._call_fanar_for_script(prompt, "opening script")
    
    def _generate_intro2_script(self, intro2_section, host_name, guest_name):
        """Generate topic introduction with guest"""
        
        content = intro2_section.get('description', '') + " " + " ".join(intro2_section.get('script', []))
        previous_context = self.context_history[-1] if self.context_history else ""
        
        prompt = f"""Generate topic introduction and guest welcome (3-4 short turns).

Topic: {content}
Previous context: {previous_context}

Style example (don't copy, use as reference only):
{self.examples['intro2']}

Requirements:
- Natural transition from previous section
- Welcome guest as expert
- Brief guest response
- Each turn: MAX 1 sentence, MAX 20 words
- Use natural transitions like: طيب، خلينا، بدل ما

Generate the script:"""
        
        return self._call_fanar_for_script(prompt, "intro2 script")
    
    def _generate_points_scripts(self, talking_points, host_name, guest_name):
        """Generate scripts for each talking point"""
        
        point_scripts = {}
        
        for i, (point_title, point_content) in enumerate(talking_points.items()):
            script = self._generate_single_point_script(
                point_title, point_content, host_name, guest_name, i
            )
            point_scripts[point_title] = script
            self.context_history.append(f"Point {i+1}: {script[:100]}...")
            print(f"✅ Generated script for point {i+1}")
        
        return point_scripts
    
    def _generate_single_point_script(self, point_title, point_content, host_name, guest_name, point_index):
        """Generate script for one talking point"""
        
        discussion = point_content.get('discussion', '')
        questions = point_content.get('questions', [])[:2]
        previous_context = self.context_history[-1] if self.context_history else ""
        
        prompt = f"""Generate dialogue for discussion point {point_index + 1} (4-5 short turns).

Point: {point_title}
Content: {discussion}
Questions: {questions}
Previous context: {previous_context}

Style example (don't copy, use as reference only):
{self.examples['point']}

Requirements:
- Host asks question, guest answers
- Quick, natural exchanges
- Each turn: MAX 1 sentence, MAX 20 words
- Use natural expressions like: ايش، بس، هنا بقى، مش بس، والله

Generate the script:"""
        
        return self._call_fanar_for_script(prompt, f"point {point_index + 1} script")
    
    def _generate_conclusion_script(self, con_section, host_name, guest_name, classification):
        """Generate conclusion script"""
        
        content = con_section.get('description', '') + " " + " ".join(con_section.get('script', []))
        full_context = " | ".join(self.context_history[-2:])
        sensitivity = classification.get('sensitivity', 'عادي')
        
        extra_note = 'أضف "والله أعلم" في النهاية' if sensitivity == 'حساس' else ''
        
        prompt = f"""اكتب خاتمة بودكاست طبيعية (2-3 دورات قصيرة).

المحتوى: {content}
الحوار السابق: {full_context}

مثال للأسلوب المطلوب (لا تنسخه، استخدمه كمرجع فقط):
{self.examples['conclusion']}

متطلبات:
- خلاصة مختصرة للموضوع
- شكر للضيف
- كل دور: جملة واحدة قصيرة
- استخدم كلمات مثل "إذن، تماماً، مش مجرد"
{extra_note}

اكتب النص:"""
        
        return self._call_fanar_for_script(prompt, "conclusion script")
    
    def _call_fanar_for_script(self, prompt, script_type):
        """Call Fanar to generate script"""
        
        system_message = """أنت خبير في كتابة حوارات البودكاست العربية الطبيعية.
اكتب حوارات قصيرة وعفوية تمزج بين الفصحى واللهجة الخليجية.
استخدم كلمات الربط والحشو الطبيعية مثل: يعني، شوي، بصراحة، طيب، ايش، بس، والله.
كل دور: جملة واحدة قصيرة وطبيعية."""
        
        try:
            response = self.deployment.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt}
                ]
            )
            
            script = response.choices[0].message.content.strip()
            return script
            
        except Exception as e:
            print(f"❌ {script_type} generation failed: {e}")
            return f"المقدم: [فشل في توليد {script_type}]"
    
    def _get_fallback_scripts(self, enhanced_outline, host_name, guest_name):
        """Simple fallback scripts"""
        
        return {
            'Intro1': """المقدم: أهلاً بكم معنا في بودكاست جديد
المقدم: موضوعنا اليوم مثير ومهم""",
            'Intro2': """المقدم: معنا اليوم ضيف متخصص
الضيف: أهلاً، شكراً للاستضافة
المقدم: أهلاً وسهلاً""",
            'Points': {"نقطة": """المقدم: ايش رايك في هالموضوع؟
الضيف: والله موضوع مهم
المقدم: زين، شكراً"""},
            'Con': """المقدم: شكراً على هالمعلومات الحلوة
الضيف: الله يعافيك
المقدم: بارك الله فيكم"""
        }
    
    def print_scripts(self, scripts):
        """Pretty print all generated scripts with time estimation"""
        
        print("\n" + "="*50)
        print("GENERATED PODCAST SCRIPTS")
        print("="*50)
        
        total_turns = 0
        
        for section, script in scripts.items():
            print(f"\n--- {section} ---")
            if isinstance(script, dict):  # Points section
                for point_title, point_script in script.items():
                    print(f"\n** {point_title} **")
                    print(point_script)
                    total_turns += len([line for line in point_script.split('\n') 
                                      if line.strip().startswith(('المقدم:', 'الضيف:'))])
            else:
                print(script)
                total_turns += len([line for line in script.split('\n') 
                                  if line.strip().startswith(('المقدم:', 'الضيف:'))])
        
        estimated_minutes = (total_turns * 3.5) / 60
        
        print(f"\n--- STATS ---")
        print(f"Total turns: {total_turns}")
        print(f"Estimated TTS time: ~{estimated_minutes:.1f} minutes")
        print("="*50)

In [19]:
script_generator = ScriptGenerator(deployment, "Fanar-C-1-8.7B")
scripts = script_generator.generate_full_script(enhanced_outline, personas, classification)
script_generator.print_scripts(scripts)

✅ Intro1 script generated
✅ Intro2 script generated
✅ Generated script for point 1
✅ Generated script for point 2
✅ Generated script for point 3
✅ Points scripts generated
✅ Conclusion script generated
✅ All scripts generated successfully!

GENERATED PODCAST SCRIPTS

--- Intro1 ---
مقدم: سلام عليكم جميعاً! إنضمّوا لنا اليوم لنغوص في عالم تقنيات الغد...
مستمع/ة محتمل: يالله، يبدو مثير حقاً!
مقدم: بالتأكيد، سنناقش تأثير الذكاء الصناعي المتزايد علينا وعلى حياتنا اليومية...

--- Intro2 ---
Intro1: مقدم: مرحبا بكم مجدداً! اليوم، نتحدث عن الذكاء الاصطناعي في التعليم...
مستمع/ة محتمل: مهتم جداً بهذا!
المقدم: طيب، دعونا نستقبل الدكتور علي، خبيراً في تكنولوجيا التعليم...
الدكتور علي: شكراً لمقدمتكم، سعيد بأن أشارك تجربتي حول الذكاء الاصطناعي في الصفوف.

--- Points ---

** النقطة الرئيسية الأولى **
مقدم: أولاً، أليس، كيف AI يستطيع فهم ما يحتاجه كل طالب بالضبط؟
أليس [الضيف]: من خلال البيانات، يتعرف على أنماط تعلم مختلفة، يعني.
مقدم: صحيح، لكن كيف هذا يفيد الطالب بشكل مباشر؟
أليس: بتوفير خطط تعليم

In [16]:
dialogue_styles = {
    "formal": {
        "host": "Good afternoon, Dr. Patel. Thank you for joining us to discuss the implications of artificial intelligence in public policy.",
        "guest": "Good afternoon, and thank you for the invitation. It’s a pleasure to contribute to this critical conversation."
    },
    "casual": {
        "host": "Hey, Alex! Great to have you on the show. We’ve got a lot to unpack about social media trends today.",
        "guest": "Hey! Super excited to be here. Let’s dive in!"
    },
    "humorous": {
        "host": "Welcome back, folks! Today we’re talking AI—yup, the thing that might steal your job, but can’t steal your coffee. Yet.",
        "guest": "Unless it learns how to queue at Starbucks, I think we’re safe—for now!"
    },
    "educational": {
        "host": "Can you break down what ‘machine learning’ actually means, for listeners who might be new to the concept?",
        "guest": "Of course! Simply put, machine learning is a way for computers to learn patterns from data, kind of like how humans learn from experience."
    },
    "inspirational": {
        "host": "You’ve overcome major obstacles to lead groundbreaking research. What kept you going through the toughest times?",
        "guest": "Honestly, the belief that what we’re building could change lives. That purpose kept me focused."
    }
}

NameError: name 'enhancer' is not defined

[{'section': 'Intro1',
  'dialogue': '**Podcast Intro Script:**\n\n🎙️ *[Upbeat music fades in]*  \n\n"Hey there, friends and listeners! 🎉 Welcome to the show! I’m Alex, and I couldn’t be more excited to have you here! This is *the* go-to place for buzzing conversations, eye-opening insights, and fresh takes on all things education. 🚀  \n\nWhether you’re an educator, a lifelong learner, or just curious about the world around you, we’ve got something special just for YOU. So get comfy, turn up the volume, and let’s dive right into today’s thought-provoking journey together. Thanks for tuning in — trust me, you don’t want to miss a second of this!"  \n\n🎙️ *[Music swells and fades out]*'},
 {'section': 'Intro2',
  'dialogue': '**Intro2 - Modified Version**\n\n🎙️ *[Upbeat and engaging music fades out]*  \n\n**Host:** Hey there, listeners! Welcome back to another exciting episode of [Podcast Name]! Buckle up, because today, we’re diving headfirst into a topic that’s transforming the world a

"Sure! Here's your engaging, conversational reworking of the provided podcast script:\n\n---\n\n🎙️ [Upbeat music transitions in]\n\n**Host Alex**: Hey there, wonderful listeners! Welcome back to another exciting episode of [Podcast Name], the place where buzzing conversations meet fresh insights and cool takes on the fascinating world of education. 🚀 \n\nI'm Alex, your podcast host and curious mind navigating the ever-evolving intersection of technology and learning. Honestly, I'm downright thrilled to have you here! So, take a deep breath, grab your favorite beverage, find a comfy spot, and let's dive right into today’s thought-provoking journey together! 🎉\n\nToday’s topic? Drumroll, please... *AI in Education*! Yes, we're going straight into the AI revolution happening in classrooms. Imagine this: a future where technology doesn’t just support, but actively empowers educators and students. Sounds a little sci-fi, doesn't it? But guess what—it's happening. Right. Now.\n\nJoining me t